<h1> FOOFing OMEGA Data </h1>

This notebook runs FOOF on MEG-PSD data.
- Option to Parallelize with Pool from Multiprocessing
- Must Specify frequency range desired for FOOF
- Saves FOOF results in pickled object

NOTES:
- For reasons unknown, parallel implementation with Pool does not currently work. 

    -It used to, but now hangs when trying to parallelize foof. 
    
- Parallel with ipyparallel is currently working!

In [1]:
# Import general code from custom module om
from __future__ import print_function

import sys
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/omegamappin/')
from om.gen import *

# Import required packages
import numpy as np


## Import packages for testing

# Import sys and add FOOF location to path
#import sys
#sys.path.append('/Users/thomasdonoghue/Documents/GitCode/')

# Import FOOF
#from foof import syn
#from foof.fit import FOOF

# Import Parallelization Packages
#from multiprocessing import Pool, freeze_support

# Import other packages
#import seaborn as sns
#import matplotlib.pyplot as plt

# Set plots inline
%matplotlib inline

In [13]:
# Get database and path details from OMDB class
db = OMDB('OMEGA')

In [15]:
sub_nums = db.check_dat_files('PSD')


Number of Subjects available: 25

Subject numbers with FOOF data available: 
[111802, 124340, 195121, 215187, 218131, 220216, 245518, 265922, 320548, 339964, 369737, 386068, 390845, 403435, 405716, 431088, 504997, 548027, 559176, 604600, 704718, 704814, 719634, 756575, 978341]



In [16]:
# Check which PSD files haven't been FOOFed
psd_files, _ = db.check_dat_files('PSD', verbose=False)
foof_files, _ = db.check_dat_files('foof', verbose=False)

not_foofed = []
for f in psd_files:
    if f not in foof_files:
        not_foofed.append(f)

print(str(len(not_foofed)), 'Subject Files not yet FOOFed: ')
print(not_foofed)

0 Subject Files not yet FOOFed: 
[]


## CHECK PSD DATA

In [17]:
# Load MEG Data
subj = 504997
psd, freqs = load_meg_psds(db.dat_source, db.psd_path, subj)

In [18]:
# Check data - get number of PSDs and frequency resolution
[nPSDs, nFreqs] = np.shape(psd)
freq_res = np.mean(np.diff(freqs))
f_min = freqs.min()
f_max = freqs.max()
print('Number of PSDs is: ', nPSDs)
print('Frequency Resolution is: ', freq_res)
print('Frequency range is: ', f_min, ' to ', f_max)

Number of PSDs is:  7500
Frequency Resolution is:  0.488281250012
Frequency range is:  0.0  to  500.000000012


In [ ]:
# Extract PSD range of interest
psd_ext, freqs_ext = extract_psd(psd, freqs, f_low=3, f_high=40)

In [ ]:
# Set up PSD as a list of 2-D np arrays
psd_list = list(psd_ext)
for i in range(0, nPSDs):
    psd_list[i] = np.reshape(psd_list[i], [len(freqs_ext), 1])

In [ ]:
# Run FOOF
#foof_results = meg_foof(psd_ext, freqs_ext, min_p=0.1, freq_res=freq_res, method='linear')

# Save out results
#save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/pickle'
#save_foof_pickle(foof_results, save_path, subj)

## Check PSDs

In [ ]:
import matplotlib.pyplot as plt

plt.plot(freqs_ext[:], psd_ext[2500, :])

### Convert pickle files (of saved FOOF results) to csv

In [ ]:
## To convert a single file

# Set subject number to convert
subj = 559176

# Get database object - set which database to use
db = OMDB('HCP')

# Load data from pickle file and resave as csv file
dat = load_foof_pickle(db.foof_path, subj)
save_foof_csv(dat, db.foof_path, subj)

In [25]:
## To convert all files
# NOTE: This will not check which files are already converted to csv, and will overwrite pre-existing csv files

# Get database object - set which database files to check
db = OMDB('HCP')

# Get list of foofed pickle files
foofed_files = db.check_files('foof', save_type='pickle', verbose=False)

# Loop through each subj, load foof from pickle, and save out as a csv
for subj in sub_nums:
    
    # Load subj foof data, from pickle file
    dat = load_foof_pickle(db.foof_path, subj)
    
    # Save out as csv file
    save_foof_csv(dat, db.foof_path, subj)

## USING POOL - (not working)

In [ ]:
# Open Pool, run FOOF in parallel, and then close pool.  - OLD
# Results stored in list of tuples"
pool = Pool(8)
results = pool.map(foof_par, psd_list[0:1])
pool.close()
pool.join()

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

# Close the pool
pool.terminate()

In [ ]:
# Parallel not working. Run linearly.  - OLD
results = [foof_par(psd) for psd in psd_list]

# Use pickle to save the results
foof_data_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/'
foof_save_name = str(subj) + '_Foof_Vertex.p'
foof_save_path = os.path.join(foof_data_path, foof_save_name)
pickle.dump(results, open(foof_save_path, 'wb'))

In [ ]:
# EXTRA: CHECK FOOF

print("model: f {0}, f_sig {1}, chi {2}".format(
        np.round(foof.centers_, decimals=1),
        np.round(foof.stdevs_, decimals=1),
        np.round(foof.chi_, decimals=1))
     )

plt.subplot(211)
plt.plot(freqs_ext, psd_example[:,0]*10000000000000000000000, label='Data')
plt.plot(*syn.mfonef(mk=foof.powers_, 
                     mf=foof.centers_, 
                     mf_sig=foof.stdevs_, 
                     f0=3, 
                     fmax=50, 
                     chi=foof.chi_, 
                     res=freq_res), 
         label='Model'
        )
plt.xlabel("F (Hz)")
plt.ylabel("PSD")
plt.yscale('log')
plt.xscale('log')
plt.legend()

## USING IPYPARALLEL

In [ ]:
# Import required things from ipyparallel
from ipyparallel import Client
from ipyparallel.util import interactive

In [ ]:
# Initialize client
c = Client()

In [ ]:
# Gather 
view = c[:]

In [ ]:
# Import required libraries to each worker
with view.sync_imports():
    import sys
    sys.path.append('/Users/thomasdonoghue/Documents/GitCode/')
    from foof.fit import FOOF

In [ ]:
# Define function to run foof
@interactive
def run_foof(psd_in):
    
    # Initialize foof object
    foof = FOOF(min_p=min_p, res=freq_res, fmin=fmin, fmax=fmax)    
    
    # Model foof
    foof.model(freqs_ext, psd_in)

    # Store vals in tuple and return
    return (foof.chi_, foof.centers_, foof.powers_, foof.stdevs_)

In [ ]:
# Send required vars to workers
view['min_p'] = 0.1
view['freq_res'] = freq_res
view['fmin'] = freqs_ext.min()
view['fmax'] = freqs_ext.max()
view['freqs_ext'] = freqs_ext

In [ ]:
%%time
# Set up and run foof parallel
foof_map = view.map(run_foof, psd_list)
foof_results = foof_map.get()

# Save out results
save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/HCP/pickle'
save_foof_pickle(foof_results, save_path, subj)

In [ ]:
min_p = 0.1
fmin = freqs_ext.min()
fmax = freqs_ext.max()

In [ ]:
%%time
out = list()
for psd_in in psd_list[0:40]:
    
    foof = FOOF(min_p=min_p, res=freq_res, fmin=fmin, fmax=fmax)
    foof.model(freqs_ext, psd_in)

    # Store vals in tuple and return
    out.append((foof.chi_, foof.centers_, foof.powers_, foof.stdevs_))

In [ ]:
44.1/23.2

In [ ]:
out[-2]

In [ ]:

out_q[-]